![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with Redisvl
## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/01_redisvl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this examples we will load a list of movie objects with the following attributes: `title`, `rating`, `description`, and `genre`. 

For the vector part of our vector search we will embed the description so that user's can search for movies that best match what they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

## Packages

In [1]:
# NBVAL_SKIP
%pip install -q redis redisvl numpy sentence-transformers

Note: you may need to restart the kernel to use updated packages.


## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [2]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [3]:
from redis import Redis

client = Redis.from_url(REDIS_URL)

In [4]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/Users/justin.cechmanek/.pyenv/versions/3.11.9/envs/redis-ai-res/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

In [7]:
movie_data[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\x9bf|=\xa4a\n;\xb6\x91\xb7;*\xcb~\xbd\x07e\xce\xbb\xc9\x16J=G\xa7?=\xcev\x95<g\xfa\x06\xbe\x0fY\xcf=+\x07p=?\xdb\r\xbd\x95\xf2H\xbdre\xc6<N\xdfa=\x958\x16\xbc\xe1\xd3\x13<N\xaa\x1c=\x13\xef\x89<\xd6\xb0-<\x8e\xb2\x9f\xbc_\x0b\xc3\xbd\xa5NR=cl\xf7\xbcP>\x17\xbe\xc0 \x05\xb9&u\xbf<0\xe2b\xba\xd6\xa6\xa8\xbdr\xdc\xec\xbcWc%=\xa6\xe7r\xbb"OG=:(\x85=s@\xa2\xbc/Z\xd0\xbdK%K\xbd\xb1\xed\x94\xbc`\xddH=\xaa&F<\xe0*\xec<\x88\xd8\x8d\xbd\xc5Z\x98<\x13\xa3\xa3=:g3\xbd+\xcd\xbd\xbd\x90$\xf7;\xf8\xf4z=\x01\xb5\x8c=\x8a\x0e\xc6\xbdoI\x90\xbd\x80\x16\xbd;u\xe7\x0c\xbd\xf32\xc9\xbc\x8b\xf8\xbb\xbcP&u\xbb9\x8f\xca<\x07\x80J=\x10\xaf*=\x96OU\xbd\xc9\xf0\x95\xbc\x10\x02\x19=\x12\xf4K<\xc0\xc2\t=L\x83\xac=\x98\xd7\xb8\xbd\xf7\xb5\x9c\xbd9\x85\x18=\x9fd&=73\xf8<\xfb\xf7\x88<\xabv\xf2\xbb%=[\xbd\xdc\xac\xee\xbb2:A\xb

## Define Redis index schema

In [8]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

index_name = "movies"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [9]:
index.info()

{'index_name': 'movies',
 'index_options': [],
 'index_definition': ['key_type',
  'HASH',
  'prefixes',
  ['rvl'],
  'default_score',
  '1'],
 'attributes': [['identifier',
   'title',
   'attribute',
   'title',
   'type',
   'TEXT',
   'WEIGHT',
   '1'],
  ['identifier',
   'description',
   'attribute',
   'description',
   'type',
   'TEXT',
   'WEIGHT',
   '1'],
  ['identifier',
   'genre',
   'attribute',
   'genre',
   'type',
   'TAG',
   'SEPARATOR',
   ',',
   'SORTABLE'],
  ['identifier',
   'rating',
   'attribute',
   'rating',
   'type',
   'NUMERIC',
   'SORTABLE',
   'UNF'],
  ['identifier',
   'vector',
   'attribute',
   'vector',
   'type',
   'VECTOR',
   'algorithm',
   'HNSW',
   'data_type',
   'FLOAT32',
   'dim',
   384,
   'distance_metric',
   'COSINE',
   'M',
   16,
   'ef_construction',
   200]],
 'num_docs': '0',
 'max_doc_id': '0',
 'num_terms': '0',
 'num_records': '0',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.02034759521484375',
 'total_inver

## Populate index

In [10]:
index.load(movie_data)

['rvl:d26b2b6be33f405eb28f6a34188640da',
 'rvl:45ada4f7810a4853bdd70cdd18522595',
 'rvl:aa41661c636c4c2f8d05a5f2c869858c',
 'rvl:fd9b27fb5e1c4f5eae98d56bed7e7181',
 'rvl:2cdd72c4e63e4627bc041bbb55e86a6d',
 'rvl:196a0596e1d54be99d327ec31e73996f',
 'rvl:9fd9753803984b2397e2ecdc29c044ca',
 'rvl:ec57dffdeec6427da6cfa8ff20c2949a',
 'rvl:69f7f09d507542b582e65653a6833708',
 'rvl:51b8e584839b42ab8070398bf7ef3924',
 'rvl:ec01492dcd004a90b1b8675d563d3c67',
 'rvl:31754d3ffbd7448b973731b2f4350796',
 'rvl:0e6827ede4eb4f889131838cf2743430',
 'rvl:8511fd326be64ef5bbdfb34ba2af7bc0',
 'rvl:c78c650c72b54266840b16b62e1cc12d',
 'rvl:a4c25db582c941f087239fd7d5337f2b',
 'rvl:6e9e9b07cc874fa28f95694ecbc3bef1',
 'rvl:fcde3ec4786f49e88ada59a934cb80bf',
 'rvl:bc3725b3d9944f4b9159412e1970cec1',
 'rvl:1fa9042adbe0444f969bd66c88f314af']

## Index loaded now we can perform vector search

### basic vector search

In [11]:
from redisvl.query import VectorQuery

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True
)

index.query(vec_query)


[{'id': 'rvl:aa41661c636c4c2f8d05a5f2c869858c',
  'vector_distance': '0.685773313046',
  'title': 'Fast & Furious 9',
  'rating': '6',
  'genre': 'action'},
 {'id': 'rvl:fcde3ec4786f49e88ada59a934cb80bf',
  'vector_distance': '0.801603198051',
  'title': 'Despicable Me',
  'rating': '7',
  'genre': 'comedy'},
 {'id': 'rvl:a4c25db582c941f087239fd7d5337f2b',
  'vector_distance': '0.812342166901',
  'title': 'The Incredibles',
  'rating': '8',
  'genre': 'comedy'}]

### Hybrid filter vector search

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

In [12]:
# Search for top 3 movies specifically in the action genre

from redisvl.query.filter import Tag

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True
)

tag_filter = Tag("genre") == "action"

vec_query.set_filter(tag_filter)

index.query(vec_query)

[{'id': 'rvl:aa41661c636c4c2f8d05a5f2c869858c',
  'vector_distance': '0.685773313046',
  'title': 'Fast & Furious 9',
  'rating': '6',
  'genre': 'action'},
 {'id': 'rvl:196a0596e1d54be99d327ec31e73996f',
  'vector_distance': '0.820429563522',
  'title': 'Mad Max: Fury Road',
  'rating': '8',
  'genre': 'action'},
 {'id': 'rvl:d26b2b6be33f405eb28f6a34188640da',
  'vector_distance': '0.851705372334',
  'title': 'Explosive Pursuit',
  'rating': '7',
  'genre': 'action'}]

In [13]:
# Search for top 3 movies specifically in the action genre with ratings at or above a 7

from redisvl.query.filter import Num

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

tag_filter = Tag("genre") == "action"
num_filter = Num("rating") >= 7
combined_filter = tag_filter & num_filter

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True,
    filter_expression=combined_filter
)

index.query(vec_query)

[{'id': 'rvl:196a0596e1d54be99d327ec31e73996f',
  'vector_distance': '0.820429563522',
  'title': 'Mad Max: Fury Road',
  'rating': '8',
  'genre': 'action'},
 {'id': 'rvl:d26b2b6be33f405eb28f6a34188640da',
  'vector_distance': '0.851705372334',
  'title': 'Explosive Pursuit',
  'rating': '7',
  'genre': 'action'},
 {'id': 'rvl:51b8e584839b42ab8070398bf7ef3924',
  'vector_distance': '0.856359839439',
  'title': 'The Avengers',
  'rating': '8',
  'genre': 'action'}]

In [14]:
# Search with full text search for movies that directly mention "criminal mastermind" in the description

from redisvl.query.filter import Text

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True
)

text_filter = Text("description") == "criminal mastermind"

vec_query.set_filter(text_filter)

index.query(vec_query)

[{'id': 'rvl:fcde3ec4786f49e88ada59a934cb80bf',
  'vector_distance': '0.801603198051',
  'title': 'Despicable Me',
  'rating': '7',
  'genre': 'comedy',
  'description': 'When a criminal mastermind uses a trio of orphan girls as pawns for a grand scheme, he finds their love is profoundly changing him for the better.'},
 {'id': 'rvl:9fd9753803984b2397e2ecdc29c044ca',
  'vector_distance': '0.98234552145',
  'title': 'The Dark Knight',
  'rating': '9',
  'genre': 'action',
  'description': 'Batman faces off against the Joker, a criminal mastermind who threatens to plunge Gotham into chaos.'}]

In [15]:
# Vector search with wildcard match

from redisvl.query.filter import Text

user_query = "High tech movies"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True
)

text_filter = Text("description") % "crim*"

vec_query.set_filter(text_filter)

index.query(vec_query)

[{'id': 'rvl:fcde3ec4786f49e88ada59a934cb80bf',
  'vector_distance': '0.801603198051',
  'title': 'Despicable Me',
  'rating': '7',
  'genre': 'comedy',
  'description': 'When a criminal mastermind uses a trio of orphan girls as pawns for a grand scheme, he finds their love is profoundly changing him for the better.'},
 {'id': 'rvl:a4c25db582c941f087239fd7d5337f2b',
  'vector_distance': '0.812342166901',
  'title': 'The Incredibles',
  'rating': '8',
  'genre': 'comedy',
  'description': "A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy."},
 {'id': 'rvl:d26b2b6be33f405eb28f

In [16]:
# Vector search with fuzzy match filter

from redisvl.query.filter import Text

user_query = "Movies with central main character"

embedded_user_query = embed_text(model, user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True
)

# Note: fuzzy match is based on Levenshtein distance. Therefore, "hero" might return result for "her" as an example.
# See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/
text_filter = Text("description") % "%hero%"

vec_query.set_filter(text_filter)

index.query(vec_query)

[{'id': 'rvl:51b8e584839b42ab8070398bf7ef3924',
  'vector_distance': '0.737778306007',
  'title': 'The Avengers',
  'rating': '8',
  'genre': 'action',
  'description': "Earth's mightiest heroes come together to stop an alien invasion that threatens the entire planet."},
 {'id': 'rvl:fd9b27fb5e1c4f5eae98d56bed7e7181',
  'vector_distance': '0.768839836121',
  'title': 'Black Widow',
  'rating': '7',
  'genre': 'action',
  'description': 'Natasha Romanoff confronts her dark past and family ties as she battles a new enemy.'},
 {'id': 'rvl:1fa9042adbe0444f969bd66c88f314af',
  'vector_distance': '0.897787868977',
  'title': 'The Princess Diaries',
  'rating': '6',
  'genre': 'comedy',
  'description': 'Mia Thermopolis has just found out that she is the heir apparent to the throne of Genovia. With her friends Lilly and Michael Moscovitz in tow, she tries to navigate through the rest of her sixteenth year.'}]

## Range queries

Range queries allow you to set a pre defined "threshold" for which we want to return documents. This is helpful when you only want documents with a certain distance from the search query.

In [17]:
from redisvl.query import RangeQuery

user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    return_score=True,
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.8
)

index.query(range_query)


[{'id': 'rvl:a4c25db582c941f087239fd7d5337f2b',
  'vector_distance': '0.644702553749',
  'title': 'The Incredibles',
  'rating': '8',
  'genre': 'comedy'},
 {'id': 'rvl:fd9b27fb5e1c4f5eae98d56bed7e7181',
  'vector_distance': '0.747987031937',
  'title': 'Black Widow',
  'rating': '7',
  'genre': 'action'},
 {'id': 'rvl:fcde3ec4786f49e88ada59a934cb80bf',
  'vector_distance': '0.750915408134',
  'title': 'Despicable Me',
  'rating': '7',
  'genre': 'comedy'},
 {'id': 'rvl:c78c650c72b54266840b16b62e1cc12d',
  'vector_distance': '0.751298904419',
  'title': 'Shrek',
  'rating': '8',
  'genre': 'comedy'},
 {'id': 'rvl:6e9e9b07cc874fa28f95694ecbc3bef1',
  'vector_distance': '0.761669456959',
  'title': 'Monsters, Inc.',
  'rating': '8',
  'genre': 'comedy'},
 {'id': 'rvl:0e6827ede4eb4f889131838cf2743430',
  'vector_distance': '0.778580009937',
  'title': 'Aladdin',
  'rating': '8',
  'genre': 'comedy'}]

Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `and` condition that returns vector search within the defined range and with a rating at or above 8.

In [18]:
user_query = "Family friendly fantasy movies"

embedded_user_query = embed_text(model, user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.7
)

numeric_filter = Num("rating") >= 8

range_query.set_filter(numeric_filter)

# in this case we want to do a simple filter search or the vector so we execute as a joint filter directly
res = index.query(range_query)

res


[{'id': 'rvl:a4c25db582c941f087239fd7d5337f2b',
  'vector_distance': '0.644702553749',
  'title': 'The Incredibles',
  'rating': '8',
  'genre': 'comedy'},
 {'id': 'rvl:c78c650c72b54266840b16b62e1cc12d',
  'vector_distance': '0.751298904419',
  'title': 'Shrek',
  'rating': '8',
  'genre': 'comedy'},
 {'id': 'rvl:6e9e9b07cc874fa28f95694ecbc3bef1',
  'vector_distance': '0.761669456959',
  'title': 'Monsters, Inc.',
  'rating': '8',
  'genre': 'comedy'},
 {'id': 'rvl:0e6827ede4eb4f889131838cf2743430',
  'vector_distance': '0.778580009937',
  'title': 'Aladdin',
  'rating': '8',
  'genre': 'comedy'}]

### Next steps

For more query examples with redisvl: [see here](https://github.com/redis/redis-vl-python/blob/main/docs/user_guide/hybrid_queries_02.ipynb)

In [19]:
# clean up!
client.flushall()

True